教程地址 https://conda.io/projects/conda/en/latest/

# Concepts
+ Conda commands
+ Conda packages
+ Conda package specification
+ Conda channels
+ Conda environments
+ Installing with conda
+ Conda performance
+ Conda for data scientists

## Conda commands
作用：
+ conda命令可以查询并搜索Anaconda包索引和当前Anaconda安装
+ 创建新的conda环境
+ 在现有的conda环境中安装和更新包  

> Tips:  
> You can abbreviate many frequently used command options that are preceded by 2 dashes (--) to just 1 dash and the first letter of the option. So --name and -n are the same, and --envs and -e are the same.

command reference https://conda.io/projects/conda/en/latest/commands.html  
Viewing command-line help https://conda.io/projects/conda/en/latest/user-guide/tasks/view-command-line-help.html 

## Conda packages
### What is a conda package?
A conda package is a compressed tarball file (.tar.bz2) or .conda file that contains:
+ system-level libraries  
+ Python or other modules  
+ executable programs and other components  
+ metadata under the info/ directory  
+ a collection of files that are installed directly into an install prefix

### .conda file format
These compressed files can be significantly smaller than their bzip2 equivalents. In addition, they decompress much more quickly. .conda is the preferred file format to use where available, although we continue to provide .tar.bz2 files in tandem.
> Note  
> In conda 4.7 and later, you cannot use package names that end in “.conda” as they conflict with the .conda file format for packages.

### Using packages
+ You may search for packages
      conda search scipy
+ You may install a packages
      conda install scipy
+ You may build a package after installing conda build
      conda build my_fun_package
   

### Package structure
    .
    ├── bin
    │   └── pyflakes
    ├── info
    │   ├── LICENSE.txt
    │   ├── files
    │   ├── index.json
    │   ├── paths.json
    │   └── recipe
    └── lib
        └── python3.5

+ bin contains relevant binaries for the package
+ lib contains the relevant library files (eg. the .py files)
+ info contains package metadata

### Metapackages
当conda包仅用于元数据不含任何文件时，它被称为元包  
An example of a metapackage is "anaconda," which collects together all the packages in the Anaconda installer. The command conda create -n envname anaconda creates an environment that exactly matches what would be created from the Anaconda installer. You can create metapackages with the conda metapackage command. Include the name and version in the command.  
+ Anaconda metapackage  
The Anaconda metapackage is used in the creation of the Anaconda Distribution installers so that they have a set of packages associated with them.  
read more https://www.anaconda.com/whats-in-a-name-clarifying-the-anaconda-metapackage/
+ Mutex metapackage  
    A mutex metapackage is a very simple package that has a name.  
    Let's look at some examples for how to use mutex metapackages to build NumPy against different BLAS implementations.
    + Building NumPy with BLAS variants  
        If you build NumPy with MKL, you also need to build SciPy, scikit-learn, and anything else using BLAS also with MKL. It is important to ensure that these “variants” (packages built with a particular set of options) are installed together and never with an alternate BLAS implementation.
    + Installing NumPy with BLAS variants
        To specify which variant of NumPy that you want, you could potentially specify the BLAS library you want:
          conda install numpy mkl
        This pathway may lead to some ambiguity and solutions with mixed BLAS, so using the metapackage is recommended. To specify MKL-powered NumPy in a non-ambiguous way, you can specify the mutex package (either directly or indirectly):
          conda install numpy “blas=*=mkl”
    + Track_features  
        One of conda’s optimization goals is to minimize the number of track_features needed to specify the desired specs. By adding track_features to one or more of the options, conda will de-prioritize it or “weigh it down.” The lowest priority package is the one that would cause the most track_features to be activated in the environment. The default package among many variants is the one that would cause the least track_features to be activated.
    + More info  
        https://github.com/AnacondaRecipes/aggregate/blob/9635228/vs2017/meta.yaml#L24

### Noarch packages
Noarch packages are packages that are not architecture specific and therefore only have to be built once. Noarch packages are either generic or Python. Noarch generic packages allow users to distribute docs, datasets, and source code in conda packages  
#### Noarch Python
Noarch Python packages cut down on the overhead of building multiple different pure Python packages on different architectures and Python versions by sorting out platform and Python version-specific differences at install time.  
In order to qualify as a noarch Python package, all of the following criteria must be fulfilled:
+ No compiled extensions
+ No post-link or pre-link or pre-unlink scripts
+ No OS-specific build scripts
+ No python version specific requirements
+ No skips except for Python version. If the recipe is py3 only, remove skip statement and add version constraint on Python in host and run section.
+ 2to3 is not used
+ Scripts argument in setup.py is not used
+ If console_script entrypoints are in setup.py, they are listed in meta.yaml
+ No activate scripts
+ Not a dependency of conda

### Link and unlink scripts
You may optionally execute scripts before and after the link and unlink steps.  
### More information
Go deeper on how to manage packages. Learn more about package metadata, repository structure and index, and package match specifications at Package specifications.

## Conda package specification
### Package metadata
The info/ directory contains all metadata about a package. Files in this location are not installed under the install prefix. Although you are free to add any file to this directory, conda only inspects the content of the files discussed below.
+ Info
    + files  
      a list of all the files in the package (not included in info/)        
    + index.json  
      metadata about the package including platform, version, dependencies, and build info  
    + paths.json  
        a list of files in the package, along with their associated SHA-256, size in bytes, and the type of path (eg. hardlink vs. softlink)
+ info/index.json  
    This file contains basic information about the package, such as name, version, build string, and dependencies.
+ info/files  
    Lists all files that are part of the package itself, 1 per line. 
+ info/has_prefix  
    Optional file.Lists all files that contain a hard-coded build prefix or placeholder prefix, which needs to be replaced by the install prefix at installation time.
+ info/license.txt  
    Optional file. The software license for the package.
+ info/no_link  
    Optional file. Lists all files that cannot be linked - either soft-linked or hard-linked - into environments and are copied instead.
+ info/about.json  
    Optional file. Contains the entries in the about section of the meta.yaml file.
+ info/recipe  
    A directory containing the full contents of the build recipe.
+ meta.yaml.rendered  
    The fully rendered build recipe. See conda render.